This example demonstrates advanced visualization techniques using HoloViews with the Bokeh plotting backend. You'll learn how to:

1. Display multiple timeseries in a single plot using `subcoordinate_y`.
2. Normalize the timeseries per *group*.
3. Create and link a minimap to the main plot with `RangeToolLink`.

Specifically, we'll simulate [Electroencephalography](https://en.wikipedia.org/wiki/Electroencephalography) (EEG) data, plot it, and then create a minimap based on the [z-score](https://en.wikipedia.org/wiki/Standard_score) of the data for easier navigation.

In [ ]:
import numpy as np
import holoviews as hv
from bokeh.models import HoverTool
from holoviews.operation.normalization import subcoordinate_group_ranges
from holoviews.plotting.links import RangeToolLink
from scipy.stats import zscore

hv.extension('bokeh')

## Generating EEG data

Let's start by simulating some EEG data. We'll create a timeseries for each channel using sine waves with varying frequencies. We'll have two groups *A* and *B* of timeseries that have different amplitudes.

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
from holoviews.operation.normalization import subcoordinate_group_ranges
from holoviews.plotting.links import RangeToolLink
from scipy.stats import zscore

hv.extension('bokeh')

GROUP_EEG = 'EEG'
GROUP_POS = 'Position'
N_CHANNELS_EEG = 10
N_CHANNELS_POS = 3
N_SECONDS = 5
SAMPLING_RATE_EEG = 200
SAMPLING_RATE_POS = 25
INIT_FREQ = 2  # Initial frequency in Hz
FREQ_INC = 5  # Frequency increment
AMPLITUDE_EEG = 100  # EEG amplitude in µV
AMPLITUDE_POS = 10  # Position amplitude in cm

# Generate time for EEG and position data
total_samples_eeg = N_SECONDS * SAMPLING_RATE_EEG
total_samples_pos = N_SECONDS * SAMPLING_RATE_POS
time_eeg = np.linspace(0, N_SECONDS, total_samples_eeg)
time_pos = np.linspace(0, N_SECONDS, total_samples_pos)

# Generate EEG timeseries data
def generate_eeg_data(index):
    return AMPLITUDE_EEG * np.sin(2 * np.pi * (INIT_FREQ + index * FREQ_INC) * time_eeg)

eeg_channels = np.arange(N_CHANNELS_EEG)
eeg_data = np.array([generate_eeg_data(i) for i in eeg_channels])
eeg_df = pd.DataFrame(eeg_data.T, index=time_eeg, columns=eeg_channels)
eeg_df.index.name = 'Time'

# Generate position data
pos_channels = ['x', 'y', 'z']
pos_data = AMPLITUDE_POS * np.random.randn(N_CHANNELS_POS, total_samples_pos).cumsum(axis=1)
pos_df = pd.DataFrame(pos_data.T, index=time_pos, columns=pos_channels)
pos_df.index.name = 'Time'

In [ ]:
# Define hover tool
hover = HoverTool(tooltips=[
    ("Info", "@info"),
    ("Time", "$x s"),
    ("Value", "$y")
])

hover_tooltips = [
    ("Type", "@info"),
    ("")
    ("Time", "$x s"),
    ("Value", "$y")
]


amp_dim = hv.Dimension("Amplitude", unit="µV")
time_dim = hv.Dimension("Time", unit="s")

# EEG data visualization
eeg_curves = []
for channel, channel_data in enumerate(eeg_data):
    info = f"EEG {channel}"
    ds = hv.Dataset((channel_data.index, channel_data, info), [time_dim, amp_dim, "Type", "Channel"])
    curve = hv.Curve(ds, time_dim, [amp_dim, "info"], group='EEG', label=info)
    curve.opts(
        subcoordinate_y=True, color="black", line_width=1, hover_tooltips=hover_tooltips,
    )
    eeg_curves.append(curve)

eeg_overlay = hv.Overlay(eeg_curves, kdims="Channel").opts(
    xlabel="Time (s)", ylabel="Channel", show_legend=False, aspect=3, responsive=True,
)

# Position data visualization
position_curves = []
for i, axis in enumerate(['x', 'y', 'z']):
    info = f"Position {axis.upper()}"
    ds = hv.Dataset((time_pos, pos_data[i], info), ["Time", "Position", "info"])
    curve = hv.Curve(ds, "Time", ["Position", "info"], group='Position', label=info)
    curve.opts(
        subcoordinate_y=True, color="blue", line_width=1, tools=[hover],
    )
    position_curves.append(curve)

position_overlay = hv.Overlay(position_curves, kdims="Axis").opts(
    xlabel="Time (s)", ylabel="Axis", show_legend=True, aspect=3, responsive=True,
)

# Combine EEG and position visualizations
combined_overlay = eeg_overlay * position_overlay

# Apply group-wise normalization
normalized_overlay = subcoordinate_group_ranges(combined_overlay)

# Display the dashboard
dashboard = normalized_overlay #opts(merge_tools=False)
dashboard

In [ ]:
GROUPS = ['A', 'B']
N_CHANNELS_PER_GROUP = {'A': 10, 'B': 5}
N_SECONDS = 5
SAMPLING_RATE = 200
INIT_FREQ = 2  # Initial frequency in Hz
FREQ_INC = 5  # Frequency increment
AMPLITUDE_PER_GROUP = {'A': 1, 'B': 10}

# Generate time and channel labels
total_samples = N_SECONDS * SAMPLING_RATE
time = np.linspace(0, N_SECONDS, total_samples)
channels = {
    group: list(range(N_CHANNELS_PER_GROUP[group]))
    for group in GROUPS
}

# Generate timeseries data
def channel_data(index, amplitude):
    return amplitude * np.sin(2 * np.pi * (INIT_FREQ + index * FREQ_INC) * time)

data = {
    group: np.array([channel_data(i, AMPLITUDE_PER_GROUP[group]) for i in range(N_CHANNELS_PER_GROUP[group])])
    for group in GROUPS
}

## Visualizing EEG Data

Next, let's dive into visualizing the EEG data. We construct each timeseries using a `Curve` element, assigning it a `group`, a `label` and setting `subcoordinate_y=True`. All these curves are then aggregated into a list, which serves as the input for an `Overlay` element. Rendering this `Overlay` produces a plot where the timeseries are stacked vertically.

Additionally, we'll enhance user interaction by implementing a custom hover tool. This will display key information—channel, time, and amplitude—when you hover over any of the curves.

In [ ]:
hover = HoverTool(tooltips=[
    ("Info", "@info"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")
])

channel_curves = []
for group in GROUPS:
    for channel, channel_data in zip(channels[group], data[group]):
        info = f"EEG {group} {channel}"
        ds = hv.Dataset((time, channel_data, info), ["Time", "Amplitude", "info"])
        curve = hv.Curve(ds, "Time", ["Amplitude", "info"], group=group, label=info)
        curve.opts(
            subcoordinate_y=True, color="black", line_width=1, tools=[hover],
        )
        channel_curves.append(curve)

eeg = hv.Overlay(channel_curves, kdims="Channel").opts(
    xlabel="Time (s)", ylabel="Channel", show_legend=False, aspect=3, responsive=True,
)
eeg

Note that the overlay above has two wheel-zoom tools. These tools are automatically attached to the groups *A* and *B*, zooming in and out is scoped to the selected group.

By default all the curves are displayed with the same *y* limits, computed over all the available data. As a consequence, timeseries in group *A*, which have a much smaller amplitude than timeseries in group *B*, appear to be quite flat and are hard to inspect. To deal with this situation, we can transform the *Overlay* with the `subcoordinate_group_ranges` operation that will apply a min-max normalization of the timeseries per group.

In [ ]:
neeg = subcoordinate_group_ranges(eeg)
neeg

## Creating the Minimap

A minimap can provide a quick overview of the data and help you navigate through it. We'll compute the z-score for each channel and represent it as an image; the z-score will normalize the data and bring out the patterns more clearly. To enable linking in the next step between the EEG `Overlay` and the minimap `Image`, we ensure they share the same y-axis range.

In [ ]:
y_positions = range(sum(N_CHANNELS_PER_GROUP.values()))
yticks = list(enumerate([f"EEG {group} {channel}" for group in channels for channel in channels[group]]))

z_data = zscore(np.concatenate(list(data.values())), axis=1)

minimap = hv.Image((time, y_positions , z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
minimap = minimap.opts(
    cmap="RdBu_r", xlabel='Time (s)', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=150, responsive=True, default_tools=[], clim=(-z_data.std(), z_data.std())
)
minimap

## Building the dashboard

Finally, we use [`RangeToolLink`](../../../user_guide/Linking_Plots.ipynb) to connect the minimap `Image` and the EEG `Overlay`, setting bounds for the initially viewable area with `boundsx` and `boundsy`, and finally a max range of 2 seconds with `intervalsx`. Once the plots are linked and assembled into a unified dashboard, you can interact with it. Experiment by dragging the selection box on the minimap or resizing it by clicking and dragging its edges.

In [ ]:
RangeToolLink(
    minimap, neeg, axes=["x", "y"],
    boundsx=(None, 2), boundsy=(None, 6.5),
    intervalsx=(None, 2),
)

dashboard = (neeg + minimap).opts(merge_tools=False).cols(1)
dashboard